# Visualize the time series of 30-m fused images from Landsat and MODIS

## A contract project of Asia Development Bank

### Contractor:
* Kaiyu Guan
* Zhan Li

In [1]:
import os
import sys
import itertools
import glob
import warnings
from collections import OrderedDict

import numpy as np
import pandas as pd
from osgeo import gdal, gdal_array, osr, ogr

import matplotlib as mpl
%matplotlib inline
import matplotlib.pyplot as plt

import seaborn as sns

In [2]:
sns.set_context("paper")
sns.set_style("whitegrid")
dpi = 300

In [3]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True) # run at the start of every ipython notebook to use plotly.offline
                     # this injects the plotly.js source files into the notebook
import plotly.tools

In [4]:
gdal.AllRegister()

In [5]:
# %%writefile ./geo_ts.py
def read_pixel_GDAL(filename, x, y):
    """ Reads in a pixel of data from an images using GDAL
    Args:
      filename (str): filename to read from
      x (int): column
      y (int): row
    Returns:
      np.ndarray: 1D array (nband) containing the pixel data
    """
    ds = gdal.Open(filename, gdal.GA_ReadOnly)
    if ds is None:
        sys.stderr.write("Failed to open the file {0:s}\n".format(filename))
        return None
    
    dtype = gdal_array.GDALTypeCodeToNumericTypeCode(
        ds.GetRasterBand(1).DataType)

    dat = np.empty(ds.RasterCount, dtype=dtype)
    for i in range(ds.RasterCount):
        dat[i] = ds.GetRasterBand(i + 1).ReadAsArray(x, y, 1, 1)
        
    ds = None

    return dat

# get time series and doy from a list of single-band image files
def get_ts_from_imgs(imgfiles, img_points):
    n = len(imgfiles)
    ts_data = {k:np.zeros(n) for k in img_points.keys()}
    for k in img_points.keys():
        for n, imgf in enumerate(imgfiles):
            ts_data[k][n] = read_pixel_GDAL(imgf, img_points[k][0], img_points[k][1])    
    # doy = np.array([int(os.path.basename(imgf).split(".")[2][4:]) for imgf in imgfiles])
    fnames = [os.path.basename(imgf) for imgf in imgfiles]
    return fnames, ts_data

def get_raster_meta_GDAL(filename, band_idx=1):
    """Retrieve the meta information of an image using GDAL
    Args:
      filename: str
          file name to read from
      band_idx: int, default 1
          index to the band to retrieve meta data for, with the first being 1.
    Returns:
      meta: dict
          returned meta data records about the image
    """
    ds = gdal.Open(filename, gdal.GA_ReadOnly)
    if ds is None:
        sys.stderr.write("Failed to open the file {0:s}\n".format(filename))
        return None
    
    dtype = gdal_array.GDALTypeCodeToNumericTypeCode(
        ds.GetRasterBand(1).DataType)
    
    band = ds.GetRasterBand(band_idx)
    nodata = band.GetNoDataValue()
    
    ncol = ds.RasterXSize
    nrow = ds.RasterYSize
    
    geo_trans = ds.GetGeoTransform()
    
    return {"DataType":dtype, "NoDataValue":nodata, "RasterXSize":ncol, "RasterYSize":nrow, \
            "GeoTransform":geo_trans}

def proj2Pixel(geoMatrix, x, y):
    """
    Uses a gdal geomatrix (gdal.GetGeoTransform()) to calculate
    the pixel location, with 0 being the first pixel, of 
    a geospatial coordinate in the projection system.
    """
    ulX = geoMatrix[0]
    ulY = geoMatrix[3]
    xDist = geoMatrix[1]
    yDist = geoMatrix[5]
    rtnX = geoMatrix[2]
    rtnY = geoMatrix[4]
    sample = int((x - ulX) / xDist)
    line = int((y - ulY) / yDist)
    return (sample, line)

def pixel2Proj(geoMatrix, sample, line):
    """
    Covnert pixel location (sample, line), 
    with 0 being the first pixel, to the 
    geospatial coordinates in the projection system, 
    with (x, y) being the UL corner of the pixel.
    """
    ulX = geoMatrix[0]
    ulY = geoMatrix[3]
    xDist = geoMatrix[1]
    yDist = geoMatrix[5]
    rtnX = geoMatrix[2]
    rtnY = geoMatrix[4]
    x = sample * xDist + ulX
    y = line * yDist + ulY
    return (x, y)

def geo2Proj(filename, lon, lat):
    """
    Convert geographic coordinates to projected coordinates 
    given the input raster file
    """
    ds = gdal.Open(filename, gdal.GA_ReadOnly)
    out_sr = osr.SpatialReference()
    out_sr.ImportFromWkt(ds.GetProjectionRef())
    in_sr = out_sr.CloneGeogCS()
    coord_trans = osr.CoordinateTransformation(in_sr, out_sr)
    return coord_trans.TransformPoint(lon, lat)[0:2]

def proj2Geo(filename, x, y):
    """
    Convert projected coordinates to geographic coordinates
    given the input raster file    
    """
    ds = gdal.Open(filename, gdal.GA_ReadOnly)
    in_sr = osr.SpatialReference()
    in_sr.ImportFromWkt(ds.GetProjectionRef())
    out_sr = in_sr.CloneGeogCS()
    coord_trans = osr.CoordinateTransformation(in_sr, out_sr)
    return coord_trans.TransformPoint(x, y)[0:2]

def geo2Pixel(filename, lon, lat):
    """
    Convert geographic coordinates to image coordinates 
    given the input raster file
    """
    ds_meta = get_raster_meta_GDAL(filename)
    proj_coord = geo2Proj(filename, lon, lat)
    return proj2Pixel(ds_meta["GeoTransform"], proj_coord[0], proj_coord[1])

def pixel2Geo(filename, sample, line):
    """
    Convert the image coordinates to geographic coordinates given the input raster file.
    """
    ds_meta = get_raster_meta_GDAL(filename)
    proj_coord = pixel2Proj(ds_meta["GeoTransform"], sample, line)
    return proj2Geo(filename, proj_coord[0], proj_coord[1])

def plot_ts(doy_list, ts_data_list, point_coords, \
            style_list=None, plot_kw_dict_list=None, xlabel="DOY", ylabel="Value", \
            figsize=(8, 4), use_plotly=False, select_doy=None, nodata=-9999):
    
    valid_flag_list = [{k:np.logical_and(ts_data[k] != nodata, ts_data[k] > 0) for k in ts_data.keys()} for ts_data in ts_data_list]
    select_keys_list = [{k:np.sum(valid_flag[k])>0 for k in valid_flag.keys()} for valid_flag in valid_flag_list]
    ymin = np.nanmin([np.nanmin([np.min(ts_data[k][valid_flag[k]]) if select_keys[k] else np.nan for k in ts_data.keys()]) \
                   for ts_data, valid_flag, select_keys in itertools.izip(ts_data_list, valid_flag_list, select_keys_list)])
    ymax = np.nanmax([np.nanmax([np.max(ts_data[k][valid_flag[k]]) if select_keys[k] else np.nan for k in ts_data.keys()]) \
                   for ts_data, valid_flag, select_keys in itertools.izip(ts_data_list, valid_flag_list, select_keys_list)])
    
    if ymin is np.nan or ymax is np.nan:
        warnings.warn("No valid values in all the given time series")
        return None
    
    colors = mpl.colors.cnames.keys()[len(doy_list)]
    
    for point_key in point_coords.keys():
        fig, ax = plt.subplots(num=point_key, figsize=figsize)
        for n, (doy, ts_data) in enumerate(itertools.izip(doy_list, ts_data_list)):
            if not select_keys_list[n][point_key]:
                continue
            flag = valid_flag_list[n][point_key]
            if style_list is not None:
                if plot_kw_dict_list is not None:
                    ax.plot(doy[flag], ts_data[point_key][flag], style_list[n], **plot_kw_dict_list[n])
                else:
                    ax.plot(doy[flag], ts_data[point_key][flag], style_list[n])
            else:
                if plot_kw_dict_list is not None:
                    ax.plot(doy[flag], ts_data[point_key][flag], **plot_kw_dict_list[n])
                else:
                    ax.plot(doy[flag], ts_data[point_key][flag])

        ax.set_ylim(ymin, ymax)
        ax.set_xlim(-1.5, 366.5)
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
        ax.set_title("{0:s}, coordinates: ({1:.6f}, {2:.6f})".format(point_key, point_coords[point_key][0], point_coords[point_key][1]))    

        if use_plotly:
            plotly_fig = plotly.tools.mpl_to_plotly(fig)
            plotly_fig['layout']['showlegend'] = True
            plotly_fig['layout']['legend'] = dict(orientation="h")
            iplot(plotly_fig)
        else:        
            plt.figure(point_key)
            if select_doy is not None:
                for d in select_doy:
                    ax.plot(np.array([d, d]), ax.get_ylim(), '--k')
            ax.legend(ncol=2, loc='upper center', bbox_to_anchor=(0.5, -0.15))        
            plt.savefig("../figures/ts_{0:s}_{1:s}.png".format(ylabel.lower(), point_key.replace(" ", "_").lower()), \
                        dpi = dpi, bbox_inches="tight", pad_inches=0.)

In [6]:
# time series file by fusion of Landsat and MODIS BEFORE TIMESAT smoothing
ts_ndvi_fused_file = "/projectnb/echidna/lidar/zhanli86/workspace/data/projects/kaiyu-adb-crop/vietnam-fusion-ts/predicted_NDVI"

# time series file by fusion of Landsat and MODIS AFTER TIMESAT smoothing
ts_ndvi_timesat_file = "/projectnb/echidna/lidar/zhanli86/workspace/data/projects/kaiyu-adb-crop/vietnam-fusion-ts/fit_NDVI"

In [6]:
# # the lat and lon of points of which time series to be visualized
# geo_points = { \
#               "Point 1":(106.378046, 20.429023), \
#               "Point 2":(106.506207, 20.461707), \
#               "Point 3":(106.46351961, 20.48002966), \
#               "Point 4":(106.46235387, 20.47841337) \
#              }
# geo_points = { \
#               "SGveg-LSATcrop": (106.07962212, 20.61375514), \
#               "SGveg-LSATbarren": (106.08106149, 20.61374614), \
#               "SGcrop-LSATcrop": (106.31273800, 20.57257159)}

# # convert the geographic coordinates of given points to image coordinates
# img_points = {pk:geo2Pixel(ts_ndvi_fused_file, geo_points[pk][0], geo_points[pk][1]) for pk in geo_points.keys()}
# proj_points = {pk:geo2Proj(ts_ndvi_fused_file, geo_points[pk][0], geo_points[pk][1]) for pk in geo_points.keys()}

In [7]:
# # randomly select several points to visualize
# npts = 4
# ts_ndvi_fused_meta = get_raster_meta_GDAL(ts_ndvi_fused_file)
# img_points = {"Point {0:d}".format(i+1):(np.random.randint(0, ts_ndvi_fused_meta["RasterXSize"]), \
#                                        np.random.randint(0, ts_ndvi_fused_meta["RasterYSize"])) for i in range(npts)}

# # the sample (column) and line (row) of the points in the image
# img_points = { \
#               "Point 1":(1094, 1192), \
#               "Point 2":(1460, 1475) \
#              }

# # Convert image coordinates to geographic and projected coordinates
# geo_points = {k:pixel2Geo(ts_ndvi_fused_file, img_points[k][0], img_points[k][1]) for k in img_points.keys()}
# proj_points = {k:pixel2Proj(get_raster_meta_GDAL(ts_ndvi_fused_file)["GeoTransform"], img_points[k][0], img_points[k][1]) for k in img_points.keys()}

## Select points from specific classes from the classification map

In [7]:
# Give classification image files
cls_img_file_list = [\
                     "/projectnb/echidna/lidar/zhanli86/workspace/data/projects/kaiyu-adb-crop/vietnam-cls/vietnam_thai_bin_cls_rf_lsat_scenes.img", \
                     "/projectnb/echidna/lidar/zhanli86/workspace/data/projects/kaiyu-adb-crop/vietnam-cls/vietnam_thai_bin_cls_rf_sg_ndvi.img"]

cls_img_label_list = [\
                      "SR_3_Scenes", \
                      "NDVI_TS_SG"]

cls_rnd_sample_shapefile = "/projectnb/echidna/lidar/zhanli86/workspace/data/projects/kaiyu-adb-crop/vietnam-cls/vietnam_thai_bin_cls_rf_lsat_scenes_rnd_samples.shp"

# read classification map
from classify_image import ImageClassifier
ic = ImageClassifier()
cls_map_profile_list = [ic.getRasterProfile(cif) for cif in cls_img_file_list]

ncls_list = [int(cls_map_profile['Metadata']['ENVI']['classes']) - 1 for cls_map_profile in cls_map_profile_list]
no_data = 0
cls_map_list = [ic.readRaster(cif)[0][0] for cif in cls_img_file_list]
cls_map_masked_list = [np.ma.masked_equal(cm, no_data) for cm in cls_map_list]

# Designate the map to use
n = 0
cls_map = cls_map_list[n]
cls_map_profile = cls_map_profile_list[n]
no_data = 0

cls_code = np.arange(int(cls_map_profile['Metadata']['ENVI']['classes'])-1, dtype=np.int8)+1
cls_name = cls_map_profile['Metadata']['ENVI']['class_names'].strip('{}').split(',')[1:]
cls_name = [cn.strip() for cn in cls_name]
cls_npix = np.array([np.sum(cls_map==cls) for cls in cls_code])
total_npix = np.sum(cls_npix)
cls_w = cls_npix / float(total_npix)
print cls_code
print cls_name
print cls_npix
print cls_w

[1 2 3 4 5 6]
['croplands', 'barren', 'urban_and_built_up', 'water_bodies', 'wetlands', 'natural_terrestrial_veg']
[1901126  105711 1204497  729353   47654    4617]
[ 0.47611971  0.02647436  0.30165531  0.18265982  0.01193451  0.00115629]


In [8]:
# Read reference data
driver = ogr.GetDriverByName("ESRI Shapefile")
data_src = driver.Open(cls_rnd_sample_shapefile, 0)
layer = data_src.GetLayer()

npts = layer.GetFeatureCount()
layer_def = layer.GetLayerDefn()
nfields = layer_def.GetFieldCount()
# get name of fields
field_names = [layer_def.GetFieldDefn(i).GetName() for i in range(nfields)]
# set up a pandas dataframe to read the feature attributes
cls_ref_df = pd.DataFrame(np.zeros((npts, nfields)), columns=field_names)

for i, feature in enumerate(layer):
    tmp_list = [feature.GetField(fd) for fd in field_names]
    cls_ref_df.iloc[i, :] = np.array([0 if item is None else np.float(item) for item in tmp_list])

data_src.Destroy()

# Fill the secondary reference labels
tmp_flag = cls_ref_df['SecRefCode'] == 0
cls_ref_df.loc[tmp_flag, 'SecRefCode'] = cls_ref_df.loc[tmp_flag, 'PriRefCode']

# Get the classification labels
cls_ref_df['ClsCode'] = cls_map[cls_ref_df['Line'].astype(int), cls_ref_df['Sample'].astype(int)]

In [9]:
img_points = OrderedDict()

# Randomly select several pixels with correct cropland classification and 
# several with wrong cropland classification
n_rnd = 6
idx_candidates = np.where(np.logical_and(cls_ref_df['ClsCode']==1, cls_ref_df['PriRefCode']==1))[0]
if n_rnd > len(idx_candidates):
    n_rnd = len(idx_candidates)
idx_rnd = np.random.choice(idx_candidates, size=n_rnd, replace=False)
for i, ir in enumerate(idx_rnd):
    img_points['Cls({0:d})-Ref({1:d})-Pts{2:02d}'.format(int(cls_ref_df.loc[ir, 'ClsCode']), int(cls_ref_df.loc[ir, 'PriRefCode']), i+1)] \
        = (int(cls_ref_df.loc[ir, 'Sample']), int(cls_ref_df.loc[ir, 'Line']))
        
n_rnd = 6
idx_candidates = np.where(np.logical_and(cls_ref_df['ClsCode']==1, cls_ref_df['PriRefCode']!=1))[0]
if n_rnd > len(idx_candidates):
    n_rnd = len(idx_candidates)
idx_rnd = np.random.choice(idx_candidates, size=n_rnd, replace=False)
for i, ir in enumerate(idx_rnd):
    img_points['Cls({0:d})-Ref({1:d})-Pts{2:02d}'.format(int(cls_ref_df.loc[ir, 'ClsCode']), int(cls_ref_df.loc[ir, 'PriRefCode']), i+1)] \
        = (int(cls_ref_df.loc[ir, 'Sample']), int(cls_ref_df.loc[ir, 'Line']))
        
n_rnd = 6
idx_candidates = np.where(np.logical_and(cls_ref_df['ClsCode']!=1, cls_ref_df['PriRefCode']==1))[0]
if n_rnd > len(idx_candidates):
    n_rnd = len(idx_candidates)
idx_rnd = np.random.choice(idx_candidates, size=n_rnd, replace=False)
for i, ir in enumerate(idx_rnd):
    img_points['Cls({0:d})-Ref({1:d})-Pts{2:02d}'.format(int(cls_ref_df.loc[ir, 'ClsCode']), int(cls_ref_df.loc[ir, 'PriRefCode']), i+1)] \
        = (int(cls_ref_df.loc[ir, 'Sample']), int(cls_ref_df.loc[ir, 'Line']))
        
n_rnd = 6
idx_candidates = np.where(reduce(np.logical_and, (cls_ref_df['ClsCode']!=1, cls_ref_df['PriRefCode']!=1, cls_ref_df['ClsCode']==cls_ref_df['PriRefCode'])))[0]
if n_rnd > len(idx_candidates):
    n_rnd = len(idx_candidates)
idx_rnd = np.random.choice(idx_candidates, size=n_rnd, replace=False)
for i, ir in enumerate(idx_rnd):
    img_points['Cls({0:d})-Ref({1:d})-Pts{2:02d}'.format(int(cls_ref_df.loc[ir, 'ClsCode']), int(cls_ref_df.loc[ir, 'PriRefCode']), i+1)] \
        = (int(cls_ref_df.loc[ir, 'Sample']), int(cls_ref_df.loc[ir, 'Line']))

In [10]:
# Convert image coordinates to geographic and projected coordinates
geo_points = OrderedDict([(k, pixel2Geo(ts_ndvi_fused_file, img_points[k][0]+0.5, img_points[k][1]+0.5)) for k in img_points.keys()])
proj_points = OrderedDict([(k, pixel2Proj(get_raster_meta_GDAL(ts_ndvi_fused_file)["GeoTransform"], img_points[k][0]+0.5, img_points[k][1]+0.5)) for k in img_points.keys()])

In [11]:
# show the points to be visualized
print img_points
print geo_points
print proj_points

OrderedDict([('Cls(1)-Ref(1)-Pts01', (735, 591)), ('Cls(1)-Ref(1)-Pts02', (1586, 123)), ('Cls(1)-Ref(1)-Pts03', (328, 1505)), ('Cls(1)-Ref(1)-Pts04', (1284, 897)), ('Cls(1)-Ref(1)-Pts05', (1581, 216)), ('Cls(1)-Ref(1)-Pts06', (968, 81)), ('Cls(1)-Ref(3)-Pts01', (1601, 1568)), ('Cls(1)-Ref(4)-Pts02', (799, 1670)), ('Cls(1)-Ref(2)-Pts03', (244, 93)), ('Cls(1)-Ref(6)-Pts04', (565, 623)), ('Cls(1)-Ref(6)-Pts05', (937, 555)), ('Cls(1)-Ref(6)-Pts06', (1267, 1225)), ('Cls(6)-Ref(1)-Pts01', (1056, 351)), ('Cls(4)-Ref(1)-Pts02', (430, 329)), ('Cls(3)-Ref(1)-Pts03', (712, 1296)), ('Cls(3)-Ref(1)-Pts04', (556, 1240)), ('Cls(3)-Ref(1)-Pts05', (610, 1227)), ('Cls(3)-Ref(1)-Pts06', (690, 488)), ('Cls(2)-Ref(2)-Pts01', (1650, 1093)), ('Cls(2)-Ref(2)-Pts02', (1867, 1455)), ('Cls(2)-Ref(2)-Pts03', (1223, 1949)), ('Cls(3)-Ref(3)-Pts04', (928, 1983)), ('Cls(3)-Ref(3)-Pts05', (876, 1300)), ('Cls(2)-Ref(2)-Pts06', (829, 136))])
OrderedDict([('Cls(1)-Ref(1)-Pts01', (106.2759246545955, 20.59276767410469)), (

## Time series from stacked STARFM+Landsat and TIMESAT DL fitting

In [12]:
ts_ndvi_fused_data = { k:read_pixel_GDAL(ts_ndvi_fused_file, img_points[k][0], img_points[k][1]) for k in img_points.keys()}
ts_ndvi_fused_doy = np.arange(len(ts_ndvi_fused_data[ts_ndvi_fused_data.keys()[0]]))+1
# sort the data points according to doy
sort_ind = np.argsort(ts_ndvi_fused_doy)
ts_ndvi_fused_data = {k:ts_ndvi_fused_data[k][sort_ind] for k in ts_ndvi_fused_data.keys()}
ts_ndvi_fused_doy = ts_ndvi_fused_doy[sort_ind]

In [13]:
ts_ndvi_timesat_data = { k:read_pixel_GDAL(ts_ndvi_timesat_file, img_points[k][0], img_points[k][1]) for k in img_points.keys()}
ts_ndvi_timesat_doy = np.arange(len(ts_ndvi_timesat_data[ts_ndvi_timesat_data.keys()[0]]))+1
# sort the data points according to doy
sort_ind = np.argsort(ts_ndvi_timesat_doy)
ts_ndvi_timesat_data = {k:ts_ndvi_timesat_data[k][sort_ind] for k in ts_ndvi_timesat_data.keys()}
ts_ndvi_timesat_doy = ts_ndvi_timesat_doy[sort_ind]

In [14]:
ts_ndvi_fused_meta = get_raster_meta_GDAL(ts_ndvi_fused_file)
# get the nodata value of the given image
if ts_ndvi_fused_meta["NoDataValue"] is None:
    nodata = -9999

* Selected DOY for image visualization
    * Around second peak: 242, 245
    * Around the trough: 186, 192
    * Last day in the year: 365, 365

## Time series from STARFM fusion

In [15]:
# red_imgfiles = glob.glob("/projectnb/echidna/lidar/zhanli86/workspace/data/projects/kaiyu-adb-crop/vietnam-starfm/Vietnam/plndsr_250.126046.2015*.red.bin")
# nir_imgfiles = glob.glob("/projectnb/echidna/lidar/zhanli86/workspace/data/projects/kaiyu-adb-crop/vietnam-starfm/Vietnam/plndsr_250.126046.2015*.nir.bin")
# ndvi_imgfiles = glob.glob("/projectnb/echidna/lidar/zhanli86/workspace/data/projects/kaiyu-adb-crop/vietnam-starfm/Vietnam/plndsr_250.126046.2015*.ndvi.bin")

# fnames, red_ts_data = get_ts_from_imgs(red_imgfiles, img_points)
# red_doy = np.array([int(imgf.split(".")[2][4:]) for imgf in fnames])
# # sort the data points according to doy
# sort_ind = np.argsort(red_doy)
# red_ts_data = {k:red_ts_data[k][sort_ind] for k in red_ts_data.keys()}
# red_doy = red_doy[sort_ind]

# fnames, nir_ts_data = get_ts_from_imgs(nir_imgfiles, img_points)
# nir_doy = np.array([int(imgf.split(".")[2][4:]) for imgf in fnames])
# # sort the data points according to doy
# sort_ind = np.argsort(nir_doy)
# nir_ts_data = {k:nir_ts_data[k][sort_ind] for k in nir_ts_data.keys()}
# nir_doy = nir_doy[sort_ind]

# fnames, ndvi_ts_data = get_ts_from_imgs(ndvi_imgfiles, img_points)
# ndvi_doy = np.array([int(imgf.split(".")[2][4:]) for imgf in fnames])
# # sort the data points according to doy
# sort_ind = np.argsort(ndvi_doy)
# ndvi_ts_data = {k:ndvi_ts_data[k][sort_ind] for k in ndvi_ts_data.keys()}
# ndvi_doy = ndvi_doy[sort_ind]

In [16]:
markers = mpl.markers.MarkerStyle().markers

In [17]:
# plot_ts([red_doy, nir_doy], [red_ts_data, nir_ts_data], geo_points, \
#         style_list=['.r', '.b'], \
#         plot_kw_dict_list=[dict(label="Red"), dict(label="NIR")], \
#         use_plotly=True)

In [18]:
# plot_ts([np.arange(len(ts_ndvi_fused_data[ts_ndvi_fused_data.keys()[0]]))+1, ndvi_doy], [ts_ndvi_fused_data, ndvi_ts_data], geo_points, \
#         style_list=['.r', '.g'], \
#         plot_kw_dict_list=[dict(label="NDVI Predicted"), dict(label="NDVI STARFM")], \
#         use_plotly=True)

## Time series from TIMESAT SG fitting to STARFM+Landsat

In [19]:
fitsg_ndvi_imgfiles = glob.glob("/projectnb/echidna/lidar/zhanli86/workspace/data/projects/kaiyu-adb-crop/vietnam-ts-sg/fitSG_NDVI_126046.2015[0-9][0-9][0-9]")

fnames, sg_ndvi_ts_data = get_ts_from_imgs(fitsg_ndvi_imgfiles, img_points)
sg_ndvi_doy = np.array([int(imgf.split(".")[1][4:]) for imgf in fnames])
# sort the data points according to doy
sort_ind = np.argsort(sg_ndvi_doy)
sg_ndvi_ts_data = {k:sg_ndvi_ts_data[k][sort_ind] for k in sg_ndvi_ts_data.keys()}
sg_ndvi_doy = sg_ndvi_doy[sort_ind]

In [ ]:
plot_ts([ts_ndvi_fused_doy, ts_ndvi_timesat_doy, sg_ndvi_doy], \
        [ts_ndvi_fused_data, ts_ndvi_timesat_data, sg_ndvi_ts_data], geo_points, \
        style_list=['.r', '-b', '-k'], \
        plot_kw_dict_list=[dict(label="NDVI Predicted (fused + LC8)"), dict(label="NDVI TIMESAT Double Logistic"), dict(label="NDVI TIMESAT SG")], \
        use_plotly=True)